<a href="https://colab.research.google.com/github/ErickCastroAlarcon/Gpu/blob/main/Proyecto_Gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto de Gpu: Fluido de SPH

In [ ]:
# Instalamos Taichi
!pip install taichi
!pip install imageio[ffmpeg]

In [ ]:
# Paquetes
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
import imageio
import taichi as ti
ti.init(arch=ti.gpu)

[Taichi] Starting on arch=cuda


## Parametros:

In [ ]:
# Número de partículas
N = 10000

# Definimos campos para partículas
pos = ti.Vector.field(3, dtype=ti.f32, shape=N)  # posiciones (x,y)
vel = ti.Vector.field(3, dtype=ti.f32, shape=N)  # velocidades
force = ti.Vector.field(3, dtype=ti.f32, shape=N)  # fuerzas
rho = ti.field(dtype=ti.f32, shape=N)             # densidad
p = ti.field(dtype=ti.f32, shape=N)               # presión

# Parametros de la grilla 3D
h = 0.05 # Radio de Kernel
cell_size = h
nx = ny = nz = int(1.0 / cell_size) + 1
num_cells = nx * ny * nz

head = ti.field(dtype=ti.i32, shape=num_cells)
next = ti.field(dtype=ti.i32, shape=N)

## Kernels:

In [ ]:
# Iniciar Particulas
@ti.kernel
def init_particulas():
    for i in range(N):
        # Coordenadas iniciales en una grilla
        x = (i % 70) * 0.01 + 0.1   # columna
        y = (i // 70) * 0.01 + 0.1  # fila
        pos[i] = ti.Vector([x, y])
        vel[i] = ti.Vector([0.0, 0.0])
        force[i] = ti.Vector([0.0, 0.0])
        rho[i] = 1.0
        p[i] = 0.0

# Crear Grilla
@ti.kernel
def crear_grilla():
    for c in range(num_cells):
      head[c] = -1
  for i in range(N):
      p = pos[i]
      ci = int(p[0] / cell_size)
      cj = int(p[1] / cell_size)
      ck = int(p[2] / cell_size)
      ci = ti.min(ti.max(ci, 0), nx - 1)
      cj = ti.min(ti.max(cj, 0), ny - 1)
      ck = ti.min(ti.max(ck, 0), nz - 1)
      cell_id = ci + cj * nx + ck * nx * ny   # mapeo lineal
      # inserción atómica al head
      nxt = ti.atomic_xchg(head[cell_id], i)  # atomic exchange
      next[i] = nxt


In [ ]:
# Construir lista de vecinos


### Densidad:
$\rho_i = \sum_j m_j W(r_{ij},h)$

In [ ]:
# Número de partículas
N = 500
pos = ti.Vector.field(2, dtype=ti.f32, shape=N)
vel = ti.Vector.field(2, dtype=ti.f32, shape=N)

@ti.kernel
def init_particles():
    for i in range(N):
        x = (i % 20) * 0.03 + 0.2
        y = (i // 20) * 0.03 + 0.5
        pos[i] = ti.Vector([x, y])
        vel[i] = ti.Vector([0.0, 0.0])

@ti.kernel
def step(dt: ti.f32):
    for i in range(N):
        vel[i].y -= 9.8 * dt   # gravedad
        pos[i] += vel[i] * dt
        if pos[i].y < 0.0:     # rebote en suelo
            pos[i].y = 0.0
            vel[i].y *= -1.0

# --------------------------
# Animación a MP4 directo
# --------------------------
init_particles()

with imageio.get_writer("particles.mp4", fps=30, codec="libx264") as writer:
    for frame in range(1000):
        step(0.016)

        # dibujar con matplotlib
        fig, ax = plt.subplots(figsize=(4, 4))
        arr = pos.to_numpy()
        ax.scatter(arr[:, 0], arr[:, 1], s=10, c="blue")
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.set_aspect("equal")
        ax.axis("off")

        fig.canvas.draw()
        w, h = fig.canvas.get_width_height()
        buf = np.frombuffer(fig.canvas.renderer.buffer_rgba(), dtype=np.uint8)
        frame = buf.reshape(h, w, 4)[..., :3]           # quitar canal alpha
        # frame tiene shape (height, width, 3) dtype uint8
        writer.append_data(frame)

        plt.close(fig)
